#### Deliverable 5: Sales Fact Table Implementation

In [110]:
import pandas as pd
import numpy as np
import sqlite3 as lite
from datetime import datetime

In [111]:
store_databases = {
    "Store 1": {
        "database_name": "store1.db",
        "transactions_table_name": "user_transactions",
    },
    "Store 2": {
        "database_name": "store2.db",
        "transactions_table_name": "sales",
    },
    "Store 3": {
        "database_name": "store3.db",
        "transactions_table_name": "user_transactions",
    },
    "Store 4": {
        "database_name": "store4.db",
        "transactions_table_name":  "sales_transactions"
    }
}

In [112]:
def standardize_date(date_str, store_name):
    """Convert various date formats to YYYYMMDD standard"""
    if pd.isna(date_str):
        return None

    if store_name == 'Store 1':
        formats = ['%Y-%m-%d']
    else:
        formats = ['%Y-%m-%d', '%y/%m/%d']

    for fmt in formats:
        try:
            dt = pd.to_datetime(date_str, format=fmt)
            return int(dt.strftime('%Y%m%d'))
        except:
            continue

    print(f"Warning: Could not parse date from {store_name}: {date_str}")
    return None

In [113]:
# Function to load December data from a store's database
def load_december_data(store):
    conn = lite.connect(store['database_name'])

    table_name = store['transactions_table_name']
    _date = 'date'
    
    # Load December transactions (filter for month = 12)
    december_transactions_query = """
        SELECT * 
        FROM """ + table_name + """ 
        WHERE """ + _date + """ LIKE '%/12/%' OR """ + _date + """ LIKE '%-12-%' OR """ + _date + """ LIKE '%12%'
    """

    print(december_transactions_query)
    december_transactions_df = pd.read_sql_query(
        december_transactions_query, conn)

    conn.close()

    return december_transactions_df

In [115]:
import sqlite3

conn = sqlite3.connect("store1.db")
date_dim = pd.read_sql("select DateKey, Date from DateDimension", conn)


In [84]:
# Main ETL Process
all_transactions = []

for store_name, config in store_databases.items():
    print(f"Processing {store_name}...")

    transactions = load_december_data(config)
    transactions['StoreKey'] = store_name.split(" ")[1]
    transactions['temp_date'] = pd.to_datetime(transactions['date'])

    transactions = transactions.merge(
        date_dim,
        left_on='temp_date',
        right_on=pd.to_datetime(date_dim['Date']),
        how='left'
    ).drop(columns=['temp_date', 'Date'])

    all_transactions.append(transactions)

Processing Store 1...

        SELECT * 
        FROM user_transactions 
        WHERE date LIKE '%/12/%' OR date LIKE '%-12-%' OR date LIKE '%12%'
    
Processing Store 2...

        SELECT * 
        FROM sales 
        WHERE date LIKE '%/12/%' OR date LIKE '%-12-%' OR date LIKE '%12%'
    
Processing Store 3...

        SELECT * 
        FROM user_transactions 
        WHERE date LIKE '%/12/%' OR date LIKE '%-12-%' OR date LIKE '%12%'
    
Processing Store 4...

        SELECT * 
        FROM sales_transactions 
        WHERE date LIKE '%/12/%' OR date LIKE '%-12-%' OR date LIKE '%12%'
    


In [85]:
combine_transactions = pd.concat(all_transactions)

C:\Users\pavan\AppData\Local\Temp\ipykernel_20948\153387707.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combine_transactions = pd.concat(all_transactions)


In [86]:
combine_transactions.head()

,transaction_id,date,customer_number,sku,sales_price,items_left,cases_ordered,StoreKey,DateKey
0,346497.0,2024/01/12,1,42358001,2.08,193,128,1,12.0
1,346498.0,2024/01/12,1,42522001,4.39,46,14,1,12.0
2,346499.0,2024/01/12,1,42314001,2.35,117,40,1,12.0
3,346500.0,2024/01/12,1,42228001,1.79,73,22,1,12.0
4,346501.0,2024/01/12,1,42884001,2.75,78,23,1,12.0


In [87]:
len(combine_transactions)

4894090

In [89]:
products = pd.read_sql("select ProductKey, SKU from ProductDimension", conn)
stores = pd.read_sql("select StoreKey from StoreDimension", conn)

#### Creating Sales Fact table (Transaction Level)

In [92]:
# Ensure StoreKey columns have the same data type
combine_transactions['StoreKey'] = combine_transactions['StoreKey'].astype(int)

fact_data = (
    combine_transactions
        .merge(products, left_on='sku', right_on="SKU", how='left')
        .merge(stores, left_on='StoreKey', right_on="StoreKey", how='left')
        .dropna(subset=['ProductKey', 'StoreKey', 'DateKey'])
)

In [94]:
fact_data.head()

,transaction_id,date,customer_number,sku,sales_price,items_left,cases_ordered,StoreKey,DateKey,ProductKey,SKU
0,346497.0,2024/01/12,1,42358001,2.08,193,128,1,12.0,365,42358001
1,346498.0,2024/01/12,1,42522001,4.39,46,14,1,12.0,533,42522001
2,346499.0,2024/01/12,1,42314001,2.35,117,40,1,12.0,306,42314001
3,346500.0,2024/01/12,1,42228001,1.79,73,22,1,12.0,197,42228001
4,346501.0,2024/01/12,1,42884001,2.75,78,23,1,12.0,935,42884001


### Calculate metrics


In [96]:
fact_data['QuantitySold'] = fact_data['cases_ordered'] * 12   #12 items per case
fact_data.head()

,transaction_id,date,customer_number,sku,sales_price,items_left,cases_ordered,StoreKey,DateKey,ProductKey,SKU,QuantitySold
0,346497.0,2024/01/12,1,42358001,2.08,193,128,1,12.0,365,42358001,1536
1,346498.0,2024/01/12,1,42522001,4.39,46,14,1,12.0,533,42522001,168
2,346499.0,2024/01/12,1,42314001,2.35,117,40,1,12.0,306,42314001,480
3,346500.0,2024/01/12,1,42228001,1.79,73,22,1,12.0,197,42228001,264
4,346501.0,2024/01/12,1,42884001,2.75,78,23,1,12.0,935,42884001,276


In [97]:
fact_data['TotalDollarSales'] = fact_data['sales_price'] * fact_data['QuantitySold']
fact_data.head()

,transaction_id,date,customer_number,sku,sales_price,items_left,cases_ordered,StoreKey,DateKey,ProductKey,SKU,QuantitySold,TotalDollarSales
0,346497.0,2024/01/12,1,42358001,2.08,193,128,1,12.0,365,42358001,1536,3194.88
1,346498.0,2024/01/12,1,42522001,4.39,46,14,1,12.0,533,42522001,168,737.52
2,346499.0,2024/01/12,1,42314001,2.35,117,40,1,12.0,306,42314001,480,1128.0
3,346500.0,2024/01/12,1,42228001,1.79,73,22,1,12.0,197,42228001,264,472.56
4,346501.0,2024/01/12,1,42884001,2.75,78,23,1,12.0,935,42884001,276,759.0


In [98]:
fact_data['TotalCostToStore'] = fact_data['sales_price'] * 0.7 * fact_data['QuantitySold']
fact_data.head()

,transaction_id,date,customer_number,sku,sales_price,items_left,cases_ordered,StoreKey,DateKey,ProductKey,SKU,QuantitySold,TotalDollarSales,TotalCostToStore
0,346497.0,2024/01/12,1,42358001,2.08,193,128,1,12.0,365,42358001,1536,3194.88,2236.416
1,346498.0,2024/01/12,1,42522001,4.39,46,14,1,12.0,533,42522001,168,737.52,516.264
2,346499.0,2024/01/12,1,42314001,2.35,117,40,1,12.0,306,42314001,480,1128.0,789.6
3,346500.0,2024/01/12,1,42228001,1.79,73,22,1,12.0,197,42228001,264,472.56,330.792
4,346501.0,2024/01/12,1,42884001,2.75,78,23,1,12.0,935,42884001,276,759.0,531.3


In [100]:
fact_data['GrossProfit'] = fact_data['TotalDollarSales'] - fact_data['TotalCostToStore']
fact_data.head()

,transaction_id,date,customer_number,sku,sales_price,items_left,cases_ordered,StoreKey,DateKey,ProductKey,SKU,QuantitySold,TotalDollarSales,TotalCostToStore,GrossProfit
0,346497.0,2024/01/12,1,42358001,2.08,193,128,1,12.0,365,42358001,1536,3194.88,2236.416,958.464
1,346498.0,2024/01/12,1,42522001,4.39,46,14,1,12.0,533,42522001,168,737.52,516.264,221.256
2,346499.0,2024/01/12,1,42314001,2.35,117,40,1,12.0,306,42314001,480,1128.0,789.6,338.4
3,346500.0,2024/01/12,1,42228001,1.79,73,22,1,12.0,197,42228001,264,472.56,330.792,141.768
4,346501.0,2024/01/12,1,42884001,2.75,78,23,1,12.0,935,42884001,276,759.0,531.3,227.7


#### Creating final fact table

In [102]:
sales_fact = fact_data[[
    'DateKey', 'customer_number', 'ProductKey', 'StoreKey',
    'QuantitySold', 'TotalDollarSales', 'TotalCostToStore', 'GrossProfit'
]].rename(columns={
    'customer_number': 'DailyCust#'
})

In [103]:
sales_fact.head()

,DateKey,DailyCust#,ProductKey,StoreKey,QuantitySold,TotalDollarSales,TotalCostToStore,GrossProfit
0,12.0,1,365,1,1536,3194.88,2236.416,958.464
1,12.0,1,533,1,168,737.52,516.264,221.256
2,12.0,1,306,1,480,1128.0,789.6,338.4
3,12.0,1,197,1,264,472.56,330.792,141.768
4,12.0,1,935,1,276,759.0,531.3,227.7


In [118]:
curr = conn.cursor()

curr.execute("DROP TABLE IF EXISTS SalesFact_TransactionLevel ")
curr.execute("""
    CREATE TABLE SalesFact_TransactionLevel (
        DateKey INT NOT NULL,
        "DailyCust#" INT NOT NULL,
        ProductKey INT NOT NULL,
        StoreKey INT NOT NULL,
        QuantitySold INT NOT NULL,
        TotalDollarSales REAL NOT NULL,
        TotalCostToStore REAL NOT NULL,
        GrossProfit REAL NOT NULL,
        PRIMARY KEY (DateKey, "DailyCust#", ProductKey),
        FOREIGN KEY (DateKey) REFERENCES DateDimension(DateKey),
        FOREIGN KEY (ProductKey) REFERENCES ProductDimension(ProductKey),
        FOREIGN KEY (StoreKey) REFERENCES StoreDimension(StoreKey)
    );
""")

In [119]:
sales_fact.to_sql("SalesFact_TransactionLevel", conn, if_exists='replace', index=False)

5592978

In [120]:
curr.close()
conn.close()